In [1]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget

In [2]:
import xarray as xr
import numpy as np
import pandas as pd

import ecephys_analyses as ea 

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
xr.set_options(keep_attrs=True)

In [4]:
def summarize(subject, condition, hyp, spws):
    count = len(spws)
    duration = hyp.duration.cumsum().max() / pd.to_timedelta(1, 's')
    rate = count / duration
    return dict(subject=subject, condition=condition, count=count, duration_secs=duration, rate_hz=rate)

In [7]:
subjects = ["Segundo", "Valentino", "Doppio", "Alessandro", "Eugene", "Allan"]

subj_dfs = list()
for subject in subjects:
    norm_hyp, norm_spws = ea.load_baseline_light_period_nrem(subject)
    dep_hyp, dep_spws = ea.load_deprivation(subject)
    early_dep_hyp, early_dep_spws = ea.load_early_deprivation(subject)
    late_dep_hyp, late_dep_spws = ea.load_late_deprivation(subject)
    early_dep_aWk_hyp, early_dep_aWk_spws = ea.load_early_deprivation(subject, states=['aWk'])
    late_dep_aWk_hyp, late_dep_aWk_spws = ea.load_late_deprivation(subject, states=['aWk'])
    early_dep_qWk_hyp, early_dep_qWk_spws = ea.load_early_deprivation(subject, states=['qWk'])
    late_dep_qWk_hyp, late_dep_qWk_spws = ea.load_late_deprivation(subject, states=['qWk'])
    early_rec_hyp, early_rec_spws = ea.load_early_recovery_nrem(subject)
    late_rec_hyp, late_rec_spws = ea.load_late_recovery_nrem(subject)
    rec_2h_hyp, rec_2h_spws = ea.load_first2h_recovery_nrem(subject)
    rec_6h_hyp, rec_6h_spws = ea.load_recovery_nrem(subject)
    bsl_2h_hyp, bsl_2h_spws = ea.load_first2h_recovery_match_nrem(subject, rec_2h_hyp)

    subj_dfs.append(
        pd.DataFrame.from_dict(
            [
                summarize(subject, "baseline-light-period-nrem", norm_hyp, norm_spws),
                summarize(subject, "recovery-first2h-match", bsl_2h_hyp, bsl_2h_spws),
                summarize(subject, "recovery-first2h", rec_2h_hyp, rec_2h_spws),
                summarize(subject, "recovery-first1h", early_rec_hyp, early_rec_spws),
                summarize(subject, "recovery-last1h", late_rec_hyp, late_rec_spws),
                summarize(subject, "recovery-full", rec_6h_hyp, rec_6h_spws),
                summarize(subject, "deprivation", dep_hyp, dep_spws),
                summarize(subject, "deprivation-first1h", early_dep_hyp, early_dep_spws),
                summarize(subject, "deprivation-last1h", late_dep_hyp, late_dep_spws),
                summarize(subject, "deprivation-first1h-aWk", early_dep_aWk_hyp, early_dep_aWk_spws),
                summarize(subject, "deprivation-last1h-aWk", late_dep_aWk_hyp, late_dep_aWk_spws),
                summarize(subject, "deprivation-first1h-qWk", early_dep_qWk_hyp, early_dep_qWk_spws),
                summarize(subject, "deprivation-last1h-qWk", late_dep_qWk_hyp, late_dep_qWk_spws)
            ]
        )
    )
    
df = pd.concat(subj_dfs)

In [8]:
df.to_csv('spw-incidence.csv', index=False)